## Configuración

In [1]:
# Cargamos paquetes
library(dplyr)
library(abind)
library(loadeR)
library(transformeR)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

The maximum JVM heap space available is: 2.00 GB

NetCDF Java Library Version: 4.6.0-SNAPSHOT (Built-On: 20150423.1338) loaded and ready

You can manually set the NetCDF Java Library version before loading the package:
options(loadeR.java_forced_version = "X.Y.Z")
library(loadeR.java)

netCDF-Java CLASSPATH from bundled java package directory: /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/lib/R/library/loadeR.java/java:/vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/lib/R/library/loadeR.java/java/netcdfAll-4.6.0-SNAPSHOT.jar

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this pa


    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.



In [2]:
# Region de estudio

lon = c(-20, 15)
lat = c(27, 47)

## Cargar datos SEAS5 (1º)

In [3]:
# HINDCAST
anios = 1981:2016
anios = anios[anios != 1995]
meses_ini = c("03")

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/tas/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_tas_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tas",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
hindcast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(hindcast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_hcst_1 = hindcast[["mes_03"]]

# Combinamos los grids en la dimensión temporal
var_hcst_1_grid = bindGrid(var_hcst_1, dimension = "time")

In [4]:
# FORECAST
anios = 2017:2022
meses_ini = c("03")  # mes de inicialización para LM = 1

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/tas/ecmwf/51/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_tas_ecmwf_51_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "tas",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_fcst_1 = forecast[["mes_03"]]

# Combinamos los grids en la dimensión temporal
var_fcst_1_grid = bindGrid(var_fcst_1, dimension = "time")

# Selecciono los primeros 25 miembros
var_fcst_1_members = subsetGrid(var_fcst_1_grid, members = 1:25)

# Combinamos los grids de hindcast y forecast
var_seas5_1 = bindGrid(var_hcst_1_grid, var_fcst_1_members, dimension = "time")

saveRDS(var_seas5_1, "tas_cgdds_seas5_downscaling.rds")

## Carga de datos de ERA5-Land (0.1º) (Predictando)

In [3]:
# Define años y meses
anios = 1981:2022
anios = anios[anios != 1995]
meses = sprintf("%02d", c(04, 05, 06, 07, 08, 09))

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio, mes) {
    yyyy = paste0(anio)
    yyyymm = paste0(anio, mes)
    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/REANALYSIS/ERA5-Land/data/Iberia/day/t2m/", yyyy, "/",
        "t2m_ERA5-Land_", yyyymm, ".nc"
    )
    
    # Carga el dataset
    data_aux = loadGridData(dataset = ruta,
                            var = "t2m",
                            lonLim = lon,
                            latLim = lat,
                            aggr.d = 'mean') %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Generamos la tabla de combinaciones lógica
combinaciones_list = lapply(anios, function(y) {
    data.frame(anio = y, mes = meses)})
combinaciones = do.call(rbind, combinaciones_list)

# Aplicar la función a cada combinación
var_era5_1_data = lapply(1:nrow(combinaciones), function(i) {
    cargar_dato(combinaciones$anio[i], combinaciones$mes[i])})

# Combinamos los grids en la dimensión temporal
var_era5_1_time = bindGrid(var_era5_1_data, dimension = "time")

# Pasamos las observaciones de Kelvin a Celsius
var_era5_1 = gridArithmetics(var_era5_1_time, 273.15, operator = "-")
attr(var_era5_1$Variable, "units") = "degC"

saveRDS(var_era5_1, "tas_cgdds_ERA5-Land.rds")

## Carga de datos de ERA5 (0.25º) (Predictores)

In [6]:
# Define años
anios = 1981:2022
anios = anios[anios != 1995]

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio) {
    ruta = file.path(
        "/lustre/gmeteo/WORK/DATA/C3S-CDS/ERA5_temp/raw/derived-era5-single-levels-daily-statistics/daily/native/sp",
        paste0("sp_derived-era5-single-levels-daily-statistics_", anio, ".nc"))
    
    data_aux = loadGridData(dataset = ruta,
                            var = "sp",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09),
                            aggr.d = 'mean') %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

lista_datos = lapply(anios, cargar_dato)

# Combinamos los grids en la dimensión temporal
era5_time = bindGrid(lista_datos, dimension = "time")

# Upscaling de la resolución de las observaciones
era5_ups = interpGrid(era5_time,
                      new.coordinates = getGrid(var_seas5_1),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings

saveRDS(era5_ups, "sp_ERA5.rds")

In [7]:
# Define años
anios = 1981:2022
anios = anios[anios != 1995]

# Función para construir la ruta y cargar los datos
cargar_dato = function(anio) {
    ruta = file.path(
        "/lustre/gmeteo/WORK/DATA/C3S-CDS/ERA5_temp/raw/derived-era5-single-levels-daily-statistics/daily/native/t2m",
        paste0("t2m_derived-era5-single-levels-daily-statistics_", anio, ".nc"))
    
    data_aux = loadGridData(dataset = ruta,
                            var = "t2m",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09),
                            aggr.d = 'mean') %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

lista_datos = lapply(anios, cargar_dato)

# Combinamos los grids en la dimensión temporal
era5_time = bindGrid(lista_datos, dimension = "time")

# Upscaling de la resolución de las observaciones
era5_ups = interpGrid(era5_time,
                      new.coordinates = getGrid(var_seas5_1),
                      method = "bilinear") %>% suppressMessages %>% suppressWarnings

# Pasamos las observaciones de Kelvin a Celsius
era5_tas = gridArithmetics(era5_ups, 273.15, operator = "-")
attr(era5_tas$Variable, "units") = "degC"

saveRDS(era5_tas, "tas_ERA5.rds")